In [4]:
import numpy as np # --- Process Data ---

def scrapetle():
    import requests
    from bs4 import BeautifulSoup
    url = 'http://celestrak.com/NORAD/elements/stations.txt'
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, "lxml")
    mytle=(soup.text[:168])
    lines = mytle.splitlines()
    return (lines)


def getorbit(mydatestr,npoints):
    npoints=npoints+6
    lonarr=[0] * npoints
    latarr=[0] * npoints
    from datetime import datetime, timedelta
    import sys, os
    import pytz
    from dateutil.parser import parse
    import ephem
    import math
    mydate=parse(mydatestr)
    for i in range(-2,npoints+2):
        one_hour = timedelta(hours=1)
        lines=scrapetle()
        # http://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/orbit/ISS/SVPOST.html
        iss = ephem.readtle(lines[0], lines[1], lines[2])
        orbitinminutes=24*60/iss._n
        minint = timedelta(minutes=orbitinminutes/npoints)
        dateinc=mydate +i*minint
        home = ephem.Observer()
        home.date = dateinc
        # Always get the latest ISS TLE data from:
        
        degrees_per_radian = 180.0 / math.pi       
        iss.compute(home)
        dist=2
        lonarr[i]=iss.sublong* degrees_per_radian
        latarr[i]=iss.sublat* degrees_per_radian
    return (lonarr, latarr)



def readvaisala (fname):
    data_file = open(fname)
    lats, lons = [], []
    magnitudes = []
    timestrings = []
    datestrings = []
    for index, line in enumerate(data_file.readlines()):
        if index > 0:
            datestrings.append((line.split(' ')[0]))
            timestrings.append((line.split(' ')[1]))
            lats.append(float(line.split(' ')[2]))
            lons.append(float(line.split(' ')[3]))
            magnitudes.append(float(line.split(' ')[4]))
    return datestrings, timestrings, lats, lons, magnitudes


   




 


def get_marker_color(magnitude):
    if np.abs(magnitude) < 200.0:
        return ('go')
    elif np.abs(magnitude) < 400.0:         
        return ('yo')     
    else:         
        return ('ro') 
    


        
%matplotlib inline

def plotvaisala (datestrings, timestrings, lats, lons, magnitudes, tbeg, tend):
    from mpl_toolkits.basemap import Basemap
    import matplotlib.pyplot as plt
    import numpy as np
    
    npoints=93
    import pytz
    utc_dt = datestrings[0].astimezone (pytz.utc)
    firstflashtimestr=utc_dt.isoformat()
    mydate=getantimeridiancrossingtime(firstflashtimestr)
    utc_dt2 = mydate.astimezone (pytz.utc)

    mydatestr=utc_dt2.isoformat()
    
    lonarr, latarr=getorbit(mydatestr, npoints)
    
    fig=plt.figure( figsize=(15, 6))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])

    minlat= (min (lats))
    maxlat= (max (lats))
    minlon= (min (lons))
    maxlon= (max (lons))
    
    
    minlat= -60
    maxlat= 60
    minlon= -179
    maxlon= 179
    
    minmag=(min (magnitudes))
    maxmag=(max (magnitudes))     
    # --- Build Map ---


    map =  Basemap(projection='merc',llcrnrlat=minlat,urcrnrlat=maxlat,\
                llcrnrlon=minlon,urcrnrlon=maxlon,lat_ts=20,resolution='c')
    map.drawcoastlines()
    map.drawcountries()
    #map.fillcontinents(color = 'gray')
    map.drawmapboundary()
    map.drawmeridians(np.arange(0, 360, 30),labels=[0,0,0,1],fontsize=10)
    map.drawparallels(np.arange(-90, 90, 30),labels=[1,0,0,0],fontsize=10)
    date = datetime.utcnow()
    date=datestrings[0]

    import pytz
    utc_dt = date.astimezone (pytz.utc)
    utc_dt = utc_dt.replace(tzinfo=None)
    CS=map.nightshade(utc_dt)

    min_marker_size = .09
    for lon, lat, mag in zip(lons, lats, magnitudes):
        x,y = map(lon, lat)
        msize = mag * min_marker_size
        marker_string = get_marker_color(mag)
        map.plot(x, y, marker_string, markersize=msize)

    x, y = map(lonarr, latarr) # forgot this line 
    map.plot(x, y, 'D-', markersize=4, linewidth=2, color='k', markerfacecolor='b') 
    plt.title("Vaisala GLD 360 data  from "+ tbeg 
              + ' to  '  + tend +' ,  nevents=' + str(len(datestrings)))
    tag = datestrings[0].strftime('%Y%m%dT%H%M%S')
    plt.savefig('vaisala'+tag+'.png')

    plt.show()

    




def getantimeridiancrossingtime(timestr):
    import math
    import time
    from datetime import datetime, timedelta
    import ephem
    import numpy as np
    import sys, os
    import pytz
    from dateutil.parser import parse
    #mydate=datetime(2016, 4, 7, 9, 10, 45)
    mydatenow=parse(timestr)
    home = ephem.Observer()
    home.date=mydatenow
    lines=scrapetle()
    # http://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/orbit/ISS/SVPOST.html
    iss = ephem.readtle(lines[0], lines[1], lines[2])
    degrees_per_radian = 180.0 / math.pi    
    iss.compute(home)

    issorbitsperday=iss._n
    issorbitalperiod=24*60/issorbitsperday


    #print ("ISS sublong" , iss.sublong)
    sublong=iss.sublong.real

    #print ("Position in degrees", sublong*180/np.pi)

    radtodeg=180/np.pi

    angletoam=(sublong+np.pi)
    angletoamdeg=angletoam*radtodeg
    #print ("Angle to AM in degrees", angletoam*radtodeg)
    timetoam=(angletoamdeg)*issorbitalperiod/360


    deltatoam=timedelta(minutes=timetoam)
    mydate=mydatenow-deltatoam


    #print ("Minutes to antimeridian" , timetoam)

    #print ("time Delta to antimeridian" , deltatoam)
    print ("Time ", mydatenow)

    print ("Antimeridian time", mydate)
    home.date = mydate
    iss.compute(home)
            
    print ("antimeridian estimate", iss.sublong)
    return (mydate)



def filtervaisala(datestrings, timestrings, lats, lons, magnitudes):
    lines=scrapetle()
    # http://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/orbit/ISS/SVPOST.html
    import ephem
    import math
    iss = ephem.readtle(lines[0], lines[1], lines[2])

    count=0
    degrees_per_radian = 180.0 / math.pi

    fildate=[]
    filtime=[]
    fillat=[]
    fillon=[]
    filmag=[]
    for i in range (0,datestrings.size):
        iss.compute(datestrings[i])
        distlat=iss.sublat*degrees_per_radian-lats[i]
        distlon=iss.sublong*degrees_per_radian-lons[i]

        asimbox=8

        #print (np.abs(distlat),(np.abs(distlon)) )
        if ( (np.abs(distlat) < asimbox) and (np.abs(distlon) < asimbox)): 
            #print (np.abs(distlat),(np.abs(distlon)) , datestrings[i])
            #print (lats[i], lons[i], iss.sublat, iss.sublong)
            count=count+1

            #print ("yes", i, count)
            filmag.append(magnitudes[i])
            fillon.append(lons[i])
            fillat.append(lats[i])
            fildate.append(datestrings[i])
            filtime.append(timestrings[i])
    print (count)   
    return (fildate, filtime, fillat, fillon, filmag)


def read_postgres_vaisala (tbeg, tend):
    import psycopg2
    import sys
    import numpy as np


    con = None

    try:

        con = psycopg2.connect(host='localhost', database='gld', user='asdcadmin', password='ASIMrocks') 
        cur = con.cursor()
        cur.execute("SET TIME ZONE 'UTC';")  # UTC 
        cur.execute("SELECT * FROM lightnings LIMIT 0")
        colnames = [desc[0] for desc in cur.description]
        cur.execute("SELECT * FROM lightnings WHERE ltime   BETWEEN '"+tbeg+"' AND '"+tend+"' ORDER BY ltime ASC, ltime_ns  LIMIT 2000000 ;")          
        data =   (cur.fetchall())

        #for row in rows:
        #    print (row)

    except :
        print ('Error could not connect to database'     )
        sys.exit(1)


    finally:

        if con:
            con.close()


    from numpy import array
    nparr = array( data )
    print ('npar.shape' ,nparr.shape)
    #id=nparr[:,0]
    datestrings=nparr[:,0]
    timestrings=nparr[:,1].astype(int)
    magnitudes=nparr[:,2].astype(float)
    lats=nparr[:,3].astype(float)
    lons=nparr[:,4].astype(float)
    
    return datestrings, timestrings, lats, lons, magnitudes




def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta

In [1]:
        

from datetime import date, datetime, timedelta
import pytz



import ephem
lines=scrapetle()
        # http://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/orbit/ISS/SVPOST.html
iss = ephem.readtle(lines[0], lines[1], lines[2])




issorbitinminutes=24*60/iss._n

delta=timedelta(minutes=issorbitinminutes)

                
start_date = datetime(2016, 5, 9, 8, 46, 27,420000, pytz.UTC)
end_date   = start_date+2*delta


for single_date in perdelta(start_date, end_date, delta):

    a=datetime.utcnow()


    mydate=(single_date.strftime("%Y-%m-%d %t"))
    tbeg=single_date
    tend=tbeg+timedelta(minutes=issorbitinminutes)
    tbegstr=tbeg.strftime("%Y-%m-%d %H:%M:%S")
    tendstr=tend.strftime("%Y-%m-%d %H:%M:%S")
    datestrings, timestrings, lats, lons, magnitudes=read_postgres_vaisala(tbegstr, tendstr)
    #datestrings[0]=datestrings[0].astimezone (pytz.utc)
    
    b=datetime.utcnow()


    print ("Time to fetch", b-a)
    
    
    #plotvaisala (datestrings, timestrings, lats, lons, magnitudes, tbegstr, tendstr)
    datestrings, timestrings, lats, lons, magnitudes=filtervaisala (datestrings, timestrings, lats, lons, magnitudes)
    plotvaisala (datestrings, timestrings, lats, lons, magnitudes,tbegstr, tendstr )
    c=datetime.utcnow()

    print ("Time to plot", c-b)

NameError: name 'scrapetle' is not defined